In [12]:
import numpy
import scipy.special
import csv
from sklearn.model_selection import KFold

class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes1, hiddennodes2, outputnodes, learningrate):

        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih += 1j * numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes1, self.inodes))

        self.whh = numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))
        self.whh = numpy.array(self.whh, ndmin=2, dtype='complex128')
        self.whh += 1j * numpy.random.normal(0.0, pow(self.hnodes1, -0.5), (self.hnodes2, self.hnodes1))

        self.who = numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who += 1j * numpy.random.normal(0.0, pow(self.hnodes2, -0.5), (self.onodes, self.hnodes2))

        self.lr = learningrate
        self.activation_function = lambda x: 1 / (1 + numpy.exp(-x))
        
    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T
        targets = numpy.array(targets_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real #+ 1j * final_outputs_imag

        output_errors = targets - final_outputs
        hidden2_errors = numpy.dot(self.who.T, output_errors)
        hidden1_errors = numpy.dot(self.whh.T, hidden2_errors)

        self.who += self.lr * numpy.dot(output_errors * (1.0 - numpy.square(final_outputs)), numpy.conj(hidden2_outputs).T)
        self.whh += self.lr * numpy.dot(hidden2_errors * (1.0 - numpy.square(hidden2_outputs)), numpy.conj(hidden1_outputs).T)
        self.wih += self.lr * numpy.dot(hidden1_errors * (1.0 - numpy.square(hidden1_outputs)), numpy.conj(inputs).T)

    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real #+ 1j * final_outputs_imag
        
        return final_outputs_real
        
        
# Create an instance of the neural network
input_nodes = 784
hidden_nodes1 = 200
hidden_nodes2 = 100
output_nodes = 10
learning_rate = 0.01

n = neuralNetwork(input_nodes, hidden_nodes1, hidden_nodes2, output_nodes, learning_rate)

# Define the number of epochs
epochs = 5

# Define the number of folds for cross-validation
num_folds = 5

# Load the data
data = []
with open('mnist_train_100.csv') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

# Convert the data to numpy arrays
data = numpy.array(data)

# Split the data into input features and labels
X = data[:, 1:].astype(float)
y = data[:, 0].astype(int)

# Perform cross-validation
kf = KFold(n_splits=num_folds)
scorecard = []

for train_index, test_index in kf.split(X):
    # Split the data into training and test sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the neural network on the training set
    for i in range(epochs):
        for j in range(len(X_train)):
            inputs = (X_train[j] / 255.0 * 0.99) + 0.01
            targets = numpy.zeros(output_nodes) + 0.01
            targets[y_train[j]] = 0.99
            n.train(inputs, targets)

    # Test the neural network on the test set
    predictions = []
    for j in range(len(X_test)):
        inputs = (X_test[j] / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        label = numpy.argmax(outputs)
        predictions.append(label)

    # Compare the predicted labels with the actual labels
    correct_predictions = (predictions == y_test).astype(int)
    scorecard.extend(correct_predictions)

# Calculate the performance metrics
accuracy = numpy.mean(scorecard)
print("Accuracy:", accuracy)


Accuracy: 0.96


In [15]:
import numpy
import scipy.special
import csv
from sklearn.model_selection import KFold

class neuralNetwork:
        
    def __init__(self, inputnodes, hiddennodes1,hiddennodes2 ,outputnodes,learningrate, r,theta1, theta2, theta3):

        self.inodes = inputnodes
        self.hnodes1 = hiddennodes1
        self.hnodes2 = hiddennodes2
        self.onodes = outputnodes
        
        self.theta1 = theta1
        self.theta2 = theta2
        self.theta3 = theta3
        self.r = r
        self.lr = learningrate
        
        self.theta1 = numpy.random.uniform(0, 2 * numpy.pi, (self.hnodes1, self.inodes))
        self.theta2 = numpy.random.uniform(0, 2 * numpy.pi, (self.hnodes2, self.hnodes1))
        self.theta3 = numpy.random.uniform(0, 2 * numpy.pi, (self.onodes, self.hnodes2))
        self.r = numpy.random.normal(0.0)
    
        self.wih = numpy.exp(1j*self.theta1)
        self.wih = numpy.array(self.wih, ndmin=2, dtype='complex128')
        self.wih *= self.r
        
        self.whh = numpy.exp(1j*self.theta2)
        self.whh = numpy.array(self.whh, ndmin=2, dtype='complex128')
        self.whh *= self.r
        
        self.who = numpy.exp(1j*self.theta3)
        self.who = numpy.array(self.who, ndmin=2, dtype='complex128')
        self.who *= self.r
        
        
        self.activation_function = lambda x: 1 / (1 + numpy.exp(-x))

        pass

    def train(self, inputs_list, targets_list):
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T
        targets = numpy.array(targets_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real 

        output_errors = targets - final_outputs
        hidden2_errors = numpy.dot(self.who.T, output_errors)
        hidden1_errors = numpy.dot(self.whh.T, hidden2_errors)

        self.who += self.lr * numpy.dot(output_errors * (1.0 - numpy.square(final_outputs)), numpy.conj(hidden2_outputs).T)
        self.whh += self.lr * numpy.dot(hidden2_errors * (1.0 - numpy.square(hidden2_outputs)), numpy.conj(hidden1_outputs).T)
        self.wih += self.lr * numpy.dot(hidden1_errors * (1.0 - numpy.square(hidden1_outputs)), numpy.conj(inputs).T)
        
        pass

    def query(self, inputs_list):
        
        inputs = numpy.array(inputs_list, dtype=complex, ndmin=2).T

        hidden1_inputs = numpy.dot(self.wih, inputs)
        hidden1_act = self.activation_function(numpy.linalg.norm(hidden1_inputs))
        hidden1_outputs_real = hidden1_act * numpy.cos(numpy.angle(hidden1_inputs))
        hidden1_outputs_imag = hidden1_act * numpy.sin(numpy.angle(hidden1_inputs))
        hidden1_outputs = hidden1_outputs_real + 1j * hidden1_outputs_imag

        hidden2_inputs = numpy.dot(self.whh, hidden1_outputs)
        hidden2_act = self.activation_function(numpy.linalg.norm(hidden2_inputs))
        hidden2_outputs_real = hidden2_act * numpy.cos(numpy.angle(hidden2_inputs))
        hidden2_outputs_imag = hidden2_act * numpy.sin(numpy.angle(hidden2_inputs))
        hidden2_outputs = hidden2_outputs_real + 1j * hidden2_outputs_imag

        final_inputs = numpy.dot(self.who, hidden2_outputs)
        final_outputs_real = self.activation_function(final_inputs.real)
        final_outputs_imag = self.activation_function((-1j * final_inputs.imag).real)
        final_outputs = final_outputs_real 
        
        return final_outputs_real
    
# Create an instance of the neural network
input_nodes = 784
hidden_nodes1 = 200
hidden_nodes2 = 100
output_nodes = 10
learning_rate = 0.01
r = 1
theta_1 = numpy.pi/4
theta_2 = numpy.pi/3
theta_3 = numpy.pi/2
n = neuralNetwork(input_nodes,hidden_nodes1,hidden_nodes2, output_nodes, learning_rate,r, theta_1, theta_2, theta_3)


# Define the number of epochs
epochs = 5

# Define the number of folds for cross-validation
num_folds = 5

# Load the data
data = []
with open('mnist_train_100.csv') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

# Convert the data to numpy arrays
data = numpy.array(data)

# Split the data into input features and labels
X = data[:, 1:].astype(float)
y = data[:, 0].astype(int)

# Perform cross-validation
kf = KFold(n_splits=num_folds)
scorecard = []

for train_index, test_index in kf.split(X):
    # Split the data into training and test sets for this fold
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Train the neural network on the training set
    for i in range(epochs):
        for j in range(len(X_train)):
            inputs = (X_train[j] / 255.0 * 0.99) + 0.01
            targets = numpy.zeros(output_nodes) + 0.01
            targets[y_train[j]] = 0.99
            n.train(inputs, targets)

    # Test the neural network on the test set
    predictions = []
    for j in range(len(X_test)):
        inputs = (X_test[j] / 255.0 * 0.99) + 0.01
        outputs = n.query(inputs)
        label = numpy.argmax(outputs)
        predictions.append(label)

    # Compare the predicted labels with the actual labels
    correct_predictions = (predictions == y_test).astype(int)
    scorecard.extend(correct_predictions)

# Calculate the performance metrics
accuracy = numpy.mean(scorecard)
print("Accuracy:", accuracy)



Accuracy: 0.93
